In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName('OlistData') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/17 13:06:59 INFO SparkEnv: Registering MapOutputTracker
25/10/17 13:06:59 INFO SparkEnv: Registering BlockManagerMaster
25/10/17 13:06:59 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
25/10/17 13:06:59 INFO SparkEnv: Registering OutputCommitCoordinator


In [2]:
hdfs_path = '/data/olist/'

In [3]:
!hadoop fs -ls /data/olist

Found 9 items
-rw-r--r--   2 sonalpriyam39 hadoop    9033957 2025-10-17 13:06 /data/olist/olist_customers_dataset.csv
-rw-r--r--   2 sonalpriyam39 hadoop   61273883 2025-10-17 13:06 /data/olist/olist_geolocation_dataset.csv
-rw-r--r--   2 sonalpriyam39 hadoop   15438671 2025-10-17 13:06 /data/olist/olist_order_items_dataset.csv
-rw-r--r--   2 sonalpriyam39 hadoop    5777138 2025-10-17 13:06 /data/olist/olist_order_payments_dataset.csv
-rw-r--r--   2 sonalpriyam39 hadoop   14451670 2025-10-17 13:06 /data/olist/olist_order_reviews_dataset.csv
-rw-r--r--   2 sonalpriyam39 hadoop   17654914 2025-10-17 13:06 /data/olist/olist_orders_dataset.csv
-rw-r--r--   2 sonalpriyam39 hadoop    2379446 2025-10-17 13:06 /data/olist/olist_products_dataset.csv
-rw-r--r--   2 sonalpriyam39 hadoop     174703 2025-10-17 13:06 /data/olist/olist_sellers_dataset.csv
-rw-r--r--   2 sonalpriyam39 hadoop       2613 2025-10-17 13:06 /data/olist/product_category_name_translation.csv


In [5]:
customers_df = spark.read.csv(hdfs_path + 'olist_customers_dataset.csv',header=True,inferSchema=True)
orders_df = spark.read.csv(hdfs_path + 'olist_orders_dataset.csv',header=True,inferSchema=True)
order_item_df = spark.read.csv(hdfs_path + 'olist_order_items_dataset.csv',header=True,inferSchema=True)
payments_df = spark.read.csv(hdfs_path + 'olist_order_payments_dataset.csv',header=True,inferSchema=True)
reviews_df = spark.read.csv(hdfs_path + 'olist_order_reviews_dataset.csv',header=True,inferSchema=True)
products_df = spark.read.csv(hdfs_path + 'olist_products_dataset.csv',header=True,inferSchema=True)
sellers_df = spark.read.csv(hdfs_path + 'olist_sellers_dataset.csv',header=True,inferSchema=True)
geolocation_df = spark.read.csv(hdfs_path + 'olist_geolocation_dataset.csv',header=True,inferSchema=True)
category_translation_df = spark.read.csv(hdfs_path + 'product_category_name_translation.csv',header=True,inferSchema=True)

In [6]:
customers_df.show()

+--------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                    9790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                    1151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|                    8775|     mogi das cruzes|            SP|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|                   13056|            campinas|            SP|
|879864dab9bc30475...|4c93744516667ad3b...|                   89254|      jaragua do sul|            SC|
|fd826e7cf63160e53...|addec96d2e059c80c...|            

# Identify Missing Values

In [7]:
from pyspark.sql.functions import *

In [8]:
def missing_numbers(df,df_name):
    print(f'Missing Values in {df_name}:')
    df.select([count(when(col(c).isNull(),1)).alias(c) for c in df.columns]).show()

In [9]:
missing_numbers(customers_df,'customer')

Missing Values in customer:


+-----------+------------------+------------------------+-------------+--------------+
|customer_id|customer_unique_id|customer_zip_code_prefix|customer_city|customer_state|
+-----------+------------------+------------------------+-------------+--------------+
|          0|                 0|                       0|            0|             0|
+-----------+------------------+------------------------+-------------+--------------+



In [10]:
missing_numbers(orders_df,'orders')

Missing Values in orders:
+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+
|order_id|customer_id|order_status|order_purchase_timestamp|order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+
|       0|          0|           0|                       0|              160|                        1783|                         2965|                            0|
+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+



In [11]:
missing_numbers(order_item_df,'order_item')

Missing Values in order_item:
+--------+-------------+----------+---------+-------------------+-----+-------------+
|order_id|order_item_id|product_id|seller_id|shipping_limit_date|price|freight_value|
+--------+-------------+----------+---------+-------------------+-----+-------------+
|       0|            0|         0|        0|                  0|    0|            0|
+--------+-------------+----------+---------+-------------------+-----+-------------+



In [12]:
missing_numbers(payments_df,'payments')

Missing Values in payments:
+--------+------------------+------------+--------------------+-------------+
|order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------+------------------+------------+--------------------+-------------+
|       0|                 0|           0|                   0|            0|
+--------+------------------+------------+--------------------+-------------+



# 2.Handle Missing Values

In [13]:
orders_df_cleaned = orders_df.na.drop(subset=['order_id','customer_id','order_status'])

In [14]:
orders_df_cleaned.show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [15]:
orders_df_cleaned = orders_df.fillna({'order_delivered_customer_date' : '9999-12-31'})

In [16]:
orders_df_cleaned.show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

# 3.Inpute Missing Values

In [17]:
payments_df.show()

+--------------------+------------------+------------+--------------------+-------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------------------+------------------+------------+--------------------+-------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|        99.33|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|
|25e8ea4e93396b6fa...|                 1| credit_card|                   1|        65.71|
|ba78997921bbcdc13...|                 1| credit_card|                   8|       107.78|
|42fdf880ba16b47b5...|                 1| credit_card|                   2|       128.45|
|298fcdf1f73eb413e...|                 1| credit_card|                   2|        96.12|
|771ee386b001f0620...|                 1| credit_card|                   1|        81.16|
|3d7239c394a212faa...|                 1| credit_card|                   3|        51.84|
|1f78449c8

In [20]:
from pyspark.ml.feature import Imputer

imputer = Imputer(inputCols=['payment_value'],outputCols=['payment_value_imputed']).setStrategy('mean')

payments_df_cleaned = imputer.fit(payments_df).transform(payments_df)

In [21]:
payments_df_cleaned.show()

+--------------------+------------------+------------+--------------------+-------------+---------------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|payment_value_imputed|
+--------------------+------------------+------------+--------------------+-------------+---------------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|        99.33|                99.33|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|                24.39|
|25e8ea4e93396b6fa...|                 1| credit_card|                   1|        65.71|                65.71|
|ba78997921bbcdc13...|                 1| credit_card|                   8|       107.78|               107.78|
|42fdf880ba16b47b5...|                 1| credit_card|                   2|       128.45|               128.45|
|298fcdf1f73eb413e...|                 1| credit_card|                   2|        96.12|               

In [24]:
payments_df_with_null = payments_df.withColumn('payment_value',when(col('payment_value')!= 99.33,col('payment_value')).otherwise(lit(None)))

In [25]:
payments_df_with_null.show()

+--------------------+------------------+------------+--------------------+-------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------------------+------------------+------------+--------------------+-------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|         NULL|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|
|25e8ea4e93396b6fa...|                 1| credit_card|                   1|        65.71|
|ba78997921bbcdc13...|                 1| credit_card|                   8|       107.78|
|42fdf880ba16b47b5...|                 1| credit_card|                   2|       128.45|
|298fcdf1f73eb413e...|                 1| credit_card|                   2|        96.12|
|771ee386b001f0620...|                 1| credit_card|                   1|        81.16|
|3d7239c394a212faa...|                 1| credit_card|                   3|        51.84|
|1f78449c8

In [26]:
from pyspark.ml.feature import Imputer

imputer = Imputer(inputCols=['payment_value'],outputCols=['payment_value_imputed']).setStrategy('median')

payments_df_cleaned = imputer.fit(payments_df_with_null).transform(payments_df_with_null)

payments_df_cleaned.show()


+--------------------+------------------+------------+--------------------+-------------+---------------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|payment_value_imputed|
+--------------------+------------------+------------+--------------------+-------------+---------------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|         NULL|                100.0|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|                24.39|
|25e8ea4e93396b6fa...|                 1| credit_card|                   1|        65.71|                65.71|
|ba78997921bbcdc13...|                 1| credit_card|                   8|       107.78|               107.78|
|42fdf880ba16b47b5...|                 1| credit_card|                   2|       128.45|               128.45|
|298fcdf1f73eb413e...|                 1| credit_card|                   2|        96.12|               

# 4.Standaridizing the Format

In [27]:
def print_schema(df,df_name):
    print(f'Schema of {df_name}:')
    df.printSchema()

In [29]:
print_schema(orders_df,'orders')

Schema of orders:
root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)



In [30]:
print_schema(customers_df,'customers')

Schema of customers:
root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)



In [31]:
print_schema(payments_df,'payments')

Schema of payments:
root
 |-- order_id: string (nullable = true)
 |-- payment_sequential: integer (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- payment_installments: integer (nullable = true)
 |-- payment_value: double (nullable = true)



In [32]:
orders_df.show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [33]:
orders_df_cleaned = orders_df_cleaned.withColumn('order_purchase_timestamp',to_date(col('order_purchase_timestamp')))

In [34]:
orders_df_cleaned.show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|              2017-10-02|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|              2018-07-24|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [38]:
payment_types = payments_df_cleaned.select('payment_type').distinct().rdd.flatMap(lambda x:x).collect()
print(payment_types)

['not_defined', 'credit_card', 'boleto', 'debit_card', 'voucher']


In [40]:
payments_df_cleaned = payments_df_cleaned.withColumn('payment_type',when(col('payment_type')=='boleto','Bank transfer').when(col('payment_type')=='credit_card','Credit Card').when(col('payment_type')=='debit_card','Debit Card').otherwise('other'))

In [41]:
Payment_types = payments_df_cleaned.select('payment_type').distinct().rdd.flatMap(lambda x:x).collect()
print(Payment_types)

['Bank transfer', 'Credit Card', 'Debit Card', 'other']


In [42]:
print_schema(customers_df,'customers')

Schema of customers:
root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)



In [43]:
customers_df.show()

+--------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                    9790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                    1151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|                    8775|     mogi das cruzes|            SP|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|                   13056|            campinas|            SP|
|879864dab9bc30475...|4c93744516667ad3b...|                   89254|      jaragua do sul|            SC|
|fd826e7cf63160e53...|addec96d2e059c80c...|            

In [44]:
customers_df_cleaned = customers_df.withColumn('customer_zip_code_prefix',col('customer_zip_code_prefix').cast('string'))

In [45]:
customers_df_cleaned.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: string (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)



# 5.Removing Duplicate Records

In [47]:
customers_df_cleaned = customers_df_cleaned.dropDuplicates(['customer_id'])

# 6. Data Transformation

In [52]:
order_with_details = orders_df_cleaned.join(order_item_df,'order_id','left').join(payments_df_cleaned,'order_id','left').join(customers_df_cleaned,'customer_id','left')

In [53]:
order_with_details.show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+-------------+--------------------+--------------------+-------------------+------+-------------+------------------+-------------+--------------------+-------------+---------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|            order_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|payment_sequential| payment_type|payment_installments|payment_value|payment_value_imputed|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------+------------------------

In [55]:
# Ordered Thing with maximum payment value

order_with_total_value = order_with_details.groupBy('order_id').agg(sum('payment_value').alias('total_order_value'))

max_payment = order_with_total_value.sort(desc('total_order_value'))

max_payment.show()

+--------------------+------------------+
|            order_id| total_order_value|
+--------------------+------------------+
|03caa2c082116e1d3...|         109312.64|
|ab14fdcfbe524636d...| 45256.00000000001|
|1b15974a0141d54e3...|44048.000000000015|
|2cc9089445046817a...|          36489.24|
|e8fa22c3673b1dd17...|30185.999999999993|
|736e1922ae60d0d6a...|          29099.52|
|9aec4e1ae90b23c7b...|           22346.6|
|71dab1155600756af...|21874.049999999996|
|912343626f370ead5...|          19457.04|
|4412d97cb2093633a...|          19174.38|
|9f5054bd9a3c71702...|18667.000000000004|
|428a2f660dc84138d...|          18384.75|
|3a213fcdfe7d98be7...|          17786.88|
|f80549a97eb203e15...|           17671.0|
|cf4659487be50c0c3...|          17069.76|
|be382a9e1ed251281...|16313.600000000002|
|637617b3ffe9e2f7a...|14963.639999999998|
|c52c7fbe316b5b9d5...|14577.569999999998|
|f60ce04ff8060152c...|14401.000000000002|
|73c8ab38f07dc9438...|14196.280000000004|
+--------------------+------------

In [62]:
## Time Calculation for delivery for each customer

delivery_time = order_with_details.select(col('customer_id'),datediff(col('order_delivered_customer_date'),col('order_purchase_timestamp')).alias('delivery_time'))
                                                                                                                                                    
delivery_time.show()                                                                                                                                                  


+--------------------+-------------+
|         customer_id|delivery_time|
+--------------------+-------------+
|9ef432eb625129730...|            8|
|9ef432eb625129730...|            8|
|9ef432eb625129730...|            8|
|b0830fb4747a6c6d2...|           14|
|41ce2a54c0b03bf34...|            9|
|f88197465ea7920ad...|           14|
|8ab97904e6daea886...|            3|
|503740e9ca751ccdd...|           17|
|ed0271e0b7da060a3...|      2915629|
|9bdf08b4b3b52b552...|           10|
|f54a9f0e6b351c431...|           10|
|31ad1d1b63eb99624...|           18|
|31ad1d1b63eb99624...|           18|
|494dded5b201313c6...|           13|
|494dded5b201313c6...|           13|
|7711cf624183d843a...|            6|
|d3e3b74c766bc6214...|           12|
|19402a48fe860416a...|            5|
|3df704f53d3f1d481...|           11|
|3b6828a50ffe54694...|           14|
+--------------------+-------------+
only showing top 20 rows



# 7. Advance Transformations

In [63]:
order_item_df.show()

+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35|  58.9|        13.29|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13| 239.9|        19.93|
|000229ec398224ef6...|            1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30| 199.0|        17.87|
|00024acbcdf0a6daa...|            1|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-15 10:10:18| 12.99|        12.79|
|00042b26cf59d7ce6...|            1|ac6c3623068f30de0...|df560393f3a51e745...|2017-02-13 13:57:51| 199.9|        18.14|
|00048cc3ae777c65d...|            1|ef92

In [68]:
## Calculating Outliers in Price

quantiles = order_item_df.approxQuantile('price',[0.01,0.99],0.0)

low_cutoff,high_cutoff = quantiles[0],quantiles[1]

low_cutoff,high_cutoff

(9.99, 890.0)

In [69]:
order_item_df.select('price').summary().show()

+-------+------------------+
|summary|             price|
+-------+------------------+
|  count|            112650|
|   mean|120.65373901471354|
| stddev|183.63392805026012|
|    min|              0.85|
|    25%|              39.9|
|    50%|             74.99|
|    75%|             134.9|
|    max|            6735.0|
+-------+------------------+



In [71]:
order_item_df_cleaned = order_item_df.filter((col('price') >=low_cutoff) & (col('price') <= high_cutoff))

In [72]:
order_item_df_cleaned.show()

+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35|  58.9|        13.29|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13| 239.9|        19.93|
|000229ec398224ef6...|            1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30| 199.0|        17.87|
|00024acbcdf0a6daa...|            1|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-15 10:10:18| 12.99|        12.79|
|00042b26cf59d7ce6...|            1|ac6c3623068f30de0...|df560393f3a51e745...|2017-02-13 13:57:51| 199.9|        18.14|
|00048cc3ae777c65d...|            1|ef92

In [75]:
products_df.show()

+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|          product_id|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|
+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|1e9e8ef04dbcff454...|           perfumaria|                 40|                       287|                 1|             225|               16|               10|              14|
|3aa071139cb16b67c...|                artes|                 44|                       276|                 1|            1000|               30|               18|              20|
|96bd76ec8810374ed...|        esporte_lazer|                 46|                       250|    

In [77]:
products_df_cleaned = products_df.withColumn('product_size_category',when(col('product_weight_g') <500,'Small') .when(col('product_weight_g').between(500,2000),'Medium').otherwise('Large'))

In [78]:
products_df_cleaned.show()

+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+---------------------+
|          product_id|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|product_size_category|
+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+---------------------+
|1e9e8ef04dbcff454...|           perfumaria|                 40|                       287|                 1|             225|               16|               10|              14|                Small|
|3aa071139cb16b67c...|                artes|                 44|                       276|                 1|            1000|               30|               18|              20|        

In [73]:
!hadoop fs -mkdir /data/olist_proc

In [74]:
order_with_details.write.mode('overwrite').parquet('/data/olist_proc/cleaned_data.parquet')

In [80]:
products_df_cleaned.write.mode('overwrite').parquet('/data/olist_proc/product_df_cleaned.parquet')

In [81]:
!hadoop fs -ls /data/olist_proc

Found 2 items
drwxr-xr-x   - root hadoop          0 2025-10-17 15:04 /data/olist_proc/cleaned_data.parquet
drwxr-xr-x   - root hadoop          0 2025-10-17 15:11 /data/olist_proc/product_df_cleaned.parquet


In [83]:
order_with_details.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: date (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- payment_sequential: integer (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- payment_installments: integer (nullable = true)
 |-- payment_value: double (nullable = true)
 |-- payment_value_imputed: double (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- custo

In [84]:
products_df_cleaned.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (nullable = true)
 |-- product_height_cm: integer (nullable = true)
 |-- product_width_cm: integer (nullable = true)
 |-- product_size_category: string (nullable = false)



In [86]:
%%sql
CREATE EXTERNAL TABLE cleaned_payments(
    product_id: STRING 
    product_category_name: STRING
    product_name_lenght: STRING
    product_description_lenght: INT
    product_photos_qty: INT 
    product_weight_g: INT 
    product_length_cm: INT 
    product_height_cm: INT 
    product_width_cm: INT 
    product_size_category: STRING
)
STORED AS PARQUET

LOCATION '/data/olist_proc/product_df_cleaned.parquet';

UsageError: Cell magic `%%sql` not found.


In [87]:
spark.stop()